# 🚀 Install, Import, and Log In

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms
from tqdm.notebook import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

### 0️⃣ Step 0: W&B 설치하기
colab에서 WandB를 사용하려면 wandb modul을 install 해야합니다.

In [2]:
%%capture
!pip install wandb --upgrade

### 1️⃣ Step 1: W&B 로그인

WandB의 web 서비스를 이용하기 위해선 log in이 필요합니다.

In [3]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

### 2️⃣ Step 2: config 설정 후 `wandb.init` 정의
이제 본격적으로 모델을 학습 시키기 전에 몇가지 준비를 하려고 합니다.  
1. hyper-parameter config 설정
2. model 학습을 위한 pipeline 정의
3. wandb.init()으로 wandb 서버와 연결
4. dataloader 정의
5. model 정의

### 1) config 설정

In [4]:
config  = {
    'epochs': 5,
    'classes':10,
    'batch_size': 128,
    'kernels': [16, 32],
    'weight_decay': 0.0005,
    'learning_rate': 1e-3,
    'dataset': 'MNIST',
    'architecture': 'CNN',
    'seed': 42
    }

### 2) model pipeline 정의

In [13]:
def model_pipeline(hyperparameters):
    wandb.init(project='test-pytorch', entity='jeena', config=hyperparameters)
      
    config = wandb.config

    model, train_loader, test_loader, criterion, optimizer = make(config)

    train(model, train_loader, criterion, optimizer, config)
    test(model, test_loader)
    return model

In [14]:
def make(config):
    train, test = get_data(train=True), get_data(train=False)
    train_loader = make_loader(train, batch_size=config.batch_size)
    test_loader = make_loader(test, batch_size=config.batch_size)

    model = ConvNet(config.kernels, config.classes).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(
        model.parameters(), lr=config.learning_rate)
    
    return model, train_loader, test_loader, criterion, optimizer

### 3) dataloader 정의

In [15]:
def get_data(slice=5, train=True):
    full_dataset = datasets.MNIST(root='./data/MNIST', train=train, 
                                    download=True,  transform=transforms.ToTensor())
    
    #  equiv to slicing with [::slice] 
    sub_dataset = torch.utils.data.Subset(full_dataset, 
                                          indices=range(0, len(full_dataset), slice))
    return sub_dataset


def make_loader(dataset, batch_size):
    loader = DataLoader(dataset=dataset,
                        batch_size=batch_size, 
                        shuffle=True,
                        pin_memory=True, num_workers=2)
    return loader

### 4) 모델 정의

In [16]:
# Conventional and convolutional neural network

class ConvNet(nn.Module):
    def __init__(self, kernels, classes=10):
        super(ConvNet, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, kernels[0], kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, kernels[1], kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7 * 7 * kernels[-1], classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

### 3️⃣ Step 3. `wandb.watch`와 `wandb.log`를 사용해서 gradients 추적하기
- **wandb.watch**는 gradient, topology와 관련된 정보를 visualization 하기 위한 코드입니다.
- **wandb.log**는 visualization 하고 싶은 정보를 넘겨줄 수 있습니다.

이 두가지 코드를 활용해서 gradient와 parameter를 시각화할 수 있습니다.   
wandb.watch는 학습하기 전 train의 앞부분에 위치 시켜줍니다.  
wandb.log는 학습 log를 출력하기 바로 전에 metric과 epoch을 입력합니다.

In [17]:
def train(model, loader, criterion, optimizer, config):
    wandb.watch(model, criterion, log="all", log_freq=10)

    # Run training and track with wandb
    total_batches = len(loader) * config.epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0
    for epoch in tqdm(range(config.epochs)):
        for batch_idx, (images, labels) in enumerate(loader):

            images, labels = images.to(device), labels.to(device)
    
            # Forward pass ➡
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass ⬅
            optimizer.zero_grad()
            loss.backward()

            # Step with optimizer
            optimizer.step()

            example_ct +=  len(images)

            # Report metrics every 25th batch
            if ((batch_idx + 1) % 25) == 0:
                train_log(loss, example_ct, epoch)

In [18]:
def train_log(loss, example_ct, epoch):
    loss = float(loss)

    # where the magic happens
    wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")

### 4️⃣ Optional Step 4:  `wandb.save`로 저장하기

In [19]:
def test(model, test_loader):
    model.eval()

    with torch.no_grad():
        correct, total = 0, 0
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            pred = outputs.max(1, keepdim = True)[1]                                       
            correct += pred.eq(labels.view_as(pred)).sum().item() 

            total = len(test_loader.dataset)

        print(f"Accuracy of the model on the {total} " +
              f"test images: {100 * correct / total}%")
        
        wandb.log({"test_accuracy": correct / total})

    # Save the model in the exchangeable ONNX format
    torch.onnx.export(model, images, "model.onnx")
    wandb.save("model.onnx")

## 🏃‍♀️ Run training and watch your metrics live on [wandb.ai](https://wandb.ai)!

In [20]:
model = model_pipeline(config)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

Extracting ./data/MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


  0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Loss after 03200 examples: 0.720
Loss after 06400 examples: 0.395
Loss after 09600 examples: 0.272
Loss after 15200 examples: 0.238
Loss after 18400 examples: 0.196
Loss after 21600 examples: 0.290
Loss after 27200 examples: 0.118
Loss after 30400 examples: 0.168
Loss after 33600 examples: 0.144
Loss after 39200 examples: 0.106
Loss after 42400 examples: 0.080
Loss after 45600 examples: 0.080
Loss after 51200 examples: 0.057
Loss after 54400 examples: 0.090
Loss after 57600 examples: 0.059
Accuracy of the model on the 2000 test images: 96.85%
